In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import cv2

import requests
from io import BytesIO
from PIL import Image
import numpy as np

from moviepy.editor import VideoFileClip

# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20, 12

from maskrcnn_benchmark.config import cfg
from predictor import COCODemo

In [ ]:
config_file = "/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml"

cfg.merge_from_file("/home/bsb2144/directpose/configs/fcos/fcos_kps_ms_training_R_50_FPN_1x.yaml")
centroid = False
if centroid:
    cfg.merge_from_list(['DATALOADER.NUM_WORKERS', '2', \
                     'DATATYPE', 'fly', \
                     'MODEL.ANIMAL_ON', 'True', \
                     'MODEL.FCOS_ON', 'False', \
                     'OUTPUT_DIR', 'training_dir/fcos_kps_ms_training_R_50_FPN_1x_drosophila1', \
                     'MODEL.WEIGHT', '/home/bsb2144/directpose/tools/training_dir/fcos_kps_ms_training_R_50_FPN_1x_drosophila_cenG13/model_final.pth',\
                     'DATASETS.TEST', "('drosophila_val_cocostyle',)",\
                     'DATASETS.TRAIN', "('drosophila_val_cocostyle', )",\
                     'MODEL.CENTROID', "False",\
                     'MODEL.FCOS.INFERENCE_TH','0.001',\
                    ])
else:
    cfg.merge_from_list(['DATALOADER.NUM_WORKERS', '2', \
                     'DATATYPE', 'fly', \
                     'MODEL.ANIMAL_ON', 'False', \
                     'MODEL.FCOS_ON', 'True', \
                     'OUTPUT_DIR', 'training_dir/fcos_kps_ms_training_R_50_FPN_1x_drosophila1', \
                     'MODEL.WEIGHT', '/home/bsb2144/directpose/tools/training_dir/fcos_kps_ms_training_R_50_FPN_1x_drosophila_G13/model_final.pth',\
                     'DATASETS.TEST', "('drosophila_val_cocostyle',)",\
                     'DATASETS.TRAIN', "('drosophila_val_cocostyle', )",\
                     'MODEL.CENTROID', "True",\
                     'MODEL.FCOS.INFERENCE_TH','0.001',\
                    ])


In [ ]:
imgsize = 1200
coco_demo = COCODemo(
    cfg,
    min_image_size=imgsize,
    confidence_threshold=0.1,
)


In [ ]:
import cv2 
def load(img_path):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    pil_image = cv2.imread(img_path)  
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")

In [ ]:
from os import listdir
from os.path import isfile, join

# mypath = '/home/bsb2144/directpose/tools/datasets/drosophila/train'
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and (f.find('DS_Store')<0)]

mypath = ''
onlyfiles = ['dro_test_frame.png','dro_test_frame1.png','dro_test_frame_crop.png']

In [ ]:
for im in onlyfiles:
    image = load(im)
    # imshow(image)
    # compute predictions
    predictions = coco_demo.run_on_opencv_image(image)
    
    filename = 'dro_labeled/'+im[:-4]+'_labeled'+str(imgsize)+'_cen'+str(centroid)+'.jpg'
    cv2.imwrite(filename, predictions[:, :, [2, 1, 0]]) 